###### Import the Tensorflow Library

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
# UNCOMMENT below lines to use GPUs or COMMENT below lines if you want to use cpu only
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

In [ ]:
import dill #library to store/restore the jupyter session
from numpy.random import seed
# seed(1)
from tensorflow import set_random_seed
# set_random_seed(2)

In [ ]:
from datetime import datetime, timedelta,time

In [ ]:
import numpy as np 
from scipy.stats import randint
import pandas as pd # data processing, .csv file i/o
import matplotlib.pyplot as plt # this is used for the plot the graph 
import seaborn as sns # used for plot interactive graph. 
from sklearn.metrics import mean_squared_error,r2_score


######  Import Deeplearning Libraries


In [ ]:
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout # regulariser
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam as adam
from keras.models import load_model

##### Load the filtered time series data

In [ ]:
data = pd.read_csv('./dataset/new_1yr_Nan.csv',
                 parse_dates={'dtx' : ['ds']}, infer_datetime_format=True, 
                 low_memory=False, na_values=['NaN','?'], index_col='dtx').interpolate()

In [ ]:
data.reset_index(inplace=True)
# data = data[['dtx','Series','Value']]
data = data[['dtx','y']]
data.columns = ['dt','y']
data['date'] = data['dt'].dt.date
data.set_index('dt',inplace = True)
# data = data[ data['Series'] == 'current'][['y','date']]

In [ ]:
data.head()

##### Load the prediction timeseries data

In [ ]:
last_week_df = pd.read_csv('./dataset/lastweek_opm.csv',sep = ';',low_memory=False,na_values=['NaN','?']).dropna()
# # convert the time back into the UK time 
# # COMMENT the below line if the time is already in UK timezone
last_week_df['Time'] = pd.to_datetime(last_week_df['Time'].astype(str).str[:-6]).sub(timedelta(minutes = 330))
# # UNCOMMENT the below line if the above linke is commented
# last_week_df['Time'] = pd.to_datetime(last_week_df['Time'])
week1 = last_week_df[ last_week_df['Series'] == '1w' ]
week2 = last_week_df[ last_week_df['Series'] == '2w' ]
curr = last_week_df[ last_week_df['Series'] == 'current' ][['Time','Value']]
curr.columns = ['dtx','y']

In [ ]:
curr.head()

In [ ]:
time_six = time(6,0,0)

##### Filter the Order Delievery Time Window

In [ ]:
week1['clean'] = week1['Time'].dt.time.apply(lambda x : x <= time_six ) 
week2['clean'] = week2['Time'].dt.time.apply(lambda x : x <= time_six ) 
curr['clean'] = curr['dtx'].dt.time.apply(lambda x : x <= time_six ) 
curr.loc[curr['clean'], 'y'] = np.nan
week1.loc[week1['clean'], 'Value'] = np.nan
week2.loc[week2['clean'], 'Value'] = np.nan


##### Do a linear interpolation on the filtered out time window

In [ ]:
curr = curr.interpolate()
week1 = week1.interpolate()
week2 = week2.interpolate()

In [ ]:
test_data = curr.copy().set_index('dtx')[['y']]
test_data.head()

In [ ]:
# # test_data = test_data[ test_data['Series'] == 'current'] [['Value']]
# test_data.reset_index(inplace = True)
# test_data['date'] = test_data['dtx'].dt.date
# test_data['holiday'] = test_data['date'].apply(lambda x : 1 if x in holiday_df.dt else 0)
# test_data.set_index('dtx',inplace = True)
# test_data = test_data[['y','holiday']]
# # test_data.columns = ['y','holiday']

In [ ]:
# plt.plot(data[:6000].reset_index()['dt'],data[:6000]['y'])
# plt.show()

In [ ]:
test_data = test_data.reset_index()[['dtx','y']] #,'holiday']]
test_data.columns = ['dt','y']#,'holiday']
# test_data.set_index('dt',inplace = True)

In [ ]:
test_datax = test_data.copy()
test_data.head()

In [ ]:
# data.plot(x = 'Value')
datax = data.reset_index()

In [ ]:
# def num_to_word(num):
#     word = ['zero',	'one','two','three','four','five','six','seven','eight','nine','ten','eleven','twelve','thirteen','fourteen','fifteen','sixteen','seventeen','eighteen','nineteen','twenty','twenty-one','twenty-two','twenty-three','twenty-four']
#     return word[num]

In [ ]:
# def extract_feat(df,label = 'ds'):
#     cdf = df
# #     cdf['date'] = df['ds'].dt.date
# #     cdf['hour'] = np.vectorize(num_to_word)(list(df[label].dt.hour))
# #     cdf['dayofweek'] = df[label].dt.weekday_name
# #     cdf['quarter'] = df[label].dt.quarter
#     cdf['month'] = df[label].dt.month_name()
# #     cdf['year'] = df[label].dt.year
# #     cdf['dayofyear'] = df[label].dt.dayofyear
# #     cdf['dayofmonth'] = df[label].dt.day
# #     cdf['weekofyear'] = df[label].dt.weekofyear
# #     return cdf[['hour','dayofweek','quarter','month','year','dayofyear','dayofmonth','weekofyear']]

In [ ]:
# extract_feat( datax, label = 'dt' )
# extract_feat( test_datax, label = 'dt' )

In [ ]:
# ohe_datax = pd.get_dummies(datax)
# ohe_test_datax = pd.get_dummies( test_datax )

In [ ]:
ohe_datax = datax.copy()
ohe_test_datax = test_datax.copy()

In [ ]:
ohe_datax

In [ ]:
# month_list = ['January', 'February', 'March', 'April', 'May', 'June', 'July','August', 'September', 'October', 'November', 'December']
# month_list = [ 'month_%s'%s for s in month_list]

In [ ]:
# col_list = ['dt','y','holiday']+month_list
# ohe_datax = ohe_datax[col_list]

In [ ]:
# print(ohe_test_datax.columns)
# month_not_available = [ month for month in month_list if month not in ohe_test_datax.columns ]
# # month_not_available
# for month in month_not_available: 
#     ohe_test_datax[month] = 0
# ohe_test_datax =  ohe_test_datax[col_list] # test and training data should have same feature list
# print(ohe_test_datax.columns) 

In [ ]:
clean_data = ohe_datax.set_index('dt')[['y']]  .interpolate()
clean_test_data = ohe_test_datax.set_index('dt')[['y']] .interpolate()

In [ ]:
resampled_data_test = clean_test_data.copy()

In [ ]:
# Inspired from https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True,target_label = None):
    n_vars = 1 #if type(data) is list else data.shape[1]
    if target_label == None:
        dff = pd.DataFrame(data) # not required for our case as the data is already a data frame
        target_label = 'var'
    else:
        dff  = data[[target_label]]
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(dff.shift(i))
        names += [ '%s(t-%d)' %(target_label,  i) ]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(dff.shift(-i))
        if i == 0:
            names += [('%s(t)' % (target_label))]
        else:
            names += [('%s(t+%d)' % (target_label, i))]
    return (cols,names)
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:

resampled_data = clean_data.copy()
resampled_data.shape

In [ ]:
yMax = resampled_data.max()['y']
yMin = resampled_data.min()['y']

In [ ]:
print('OPM range for training: [ %.2f, %.2f]'%(yMin, yMax))

#### normalize the training and testing data

In [ ]:
resampled_data['y']=(resampled_data['y'] - yMin )/( yMax - yMin )

In [ ]:
resampled_data_test['y'] = (resampled_data_test['y'] - yMin) / (yMax - yMin)

``pred_next``: time window to predict (in mins)<br />
``last_n``: time window used to predict  ```ped_next```<sup>th</sup> mins

In [ ]:
last_n = 120
pred_next = 30

###### Make feature set for training and testing purpose

In [ ]:
cols_train,names_train = series_to_supervised(resampled_data, n_in = (last_n+pred_next) ,n_out= 1,target_label = 'y')
cols_test,names_test = series_to_supervised(resampled_data_test, n_in = (last_n+pred_next) ,n_out= 1,target_label = 'y')

In [ ]:
# print(names_test)
test_feat = pd.concat(cols_test,axis  = 1 )
test_feat.columns = names_test
train_feat = pd.concat(cols_train,axis  = 1 )
train_feat.columns = names_train

In [ ]:
reframed_data_test = pd.concat([resampled_data_test, test_feat ],axis = 1 ).drop(columns = ['y'] ).dropna()
reframed_data = pd.concat([resampled_data, train_feat ],axis = 1 ).drop(columns = ['y'] ).dropna()

In [ ]:
# print(reframed_data_test.head())
test_date = reframed_data_test.reset_index()[['dt']]

In [ ]:
# test_date.head()

In [ ]:
dates_to_predict = list(test_date['dt'])

###### split Test and Train Data into input and outputs

In [ ]:
test = reframed_data_test.values
test_X,test_y = test[:,:-pred_next],test[:,-1]
test_X = test_X.reshape((test_X.shape[0],1,test_X.shape[1]))

In [ ]:
values = reframed_data.values
train_index = len(values) # Consume the entire training data
train = values
train_X, train_y = train[:, :-pred_next], train[:, -1]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape) 
# We reshaped the input into the 3D format as expected by LSTMs, namely [samples, timesteps, features].

In [ ]:
reframed_data.head()

In [ ]:
train_ds =  reframed_data[:train_index].reset_index()
test_ds  =  test_data.reset_index()

#### Instantiate the model here

In [ ]:
model = Sequential()
model.add(LSTM(200, return_sequences=True, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dropout(rate = 0.22))
model.add(LSTM(80))
model.add(Dropout(rate = 0.12))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

##### Model Details

In [ ]:
model.summary()

#### Model checkpointing

In [ ]:
filepath="./dataset/weights-improvement-{epoch:02d}-{val_loss:.7f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

## Train the Model using the training dataset <br />
Should be ignored if planning to load the model

In [ ]:
history = model.fit(train_X, train_y, epochs=30, batch_size=120,validation_data=(test_X, test_y),callbacks=callbacks_list, verbose=1, shuffle=False)

#### Alternatively one can save the entire session using dill
[https://dill.readthedocs.io/en/latest/dill.html#dill._dill.dump_session]

#### Training and Testing MSE  vs Epoch Numbers
<br /> Should be ignored if planning to load the model

In [ ]:
# #summarize history for Normalised MSE Loss 
plt.grid()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

#### Should not be executed if using ```model.fit```
``load_model_file`` : File name where the model was stored(``.hdf5`` file) 

In [ ]:
# load_model_file = 'change_this___with_the_model_file_name'

In [ ]:
# model = load_model( load_model_file )

#### Generate the OPM using testing data and Calculate Errors
``rmse`` : Root-Mean-Squared Error
``mape`` : Mean-Absolute Percentage Error

In [ ]:
#load_path = './dataset/weights-improvement-04-0.00.hdf5'
# model.load_weights(load_path) # UNCOMMENT to load the model from the file
yhat = model.predict(test_X, verbose=0)
rmse = np.sqrt(mean_squared_error(test_y, yhat))
metric_df = pd.DataFrame({'y':test_y,'yhat':yhat[:,0]})
metric_df = metric_df[ metric_df['y'] != 0 ] # can drop zero values (due to system crash)
c_y = np.asarray(metric_df.y)*(yMax-yMin) + yMin
c_yhat = np.asarray(metric_df.yhat)*(yMax-yMin) + yMin
mape  = np.mean(np.abs(c_y-c_yhat)/c_y)*100
print('Test RMSE: %.3f' % (rmse*(yMax-yMin)+yMin))
print('Test MAPE: %.3f'% mape)
print('Accuracy on Testing Dataset: %.3f'%(100-mape))

In [ ]:
# # UNCOMMENT to check the training-related accuracy and errors
# yhat_train = model.predict(train_X, verbose=0)
# rmse_train = np.sqrt(mean_squared_error(train_y, yhat_train))
# metric_df_train = pd.DataFrame({'y':train_y,'yhat':yhat_train[:,0]})
# metric_df_train = metric_df_train[ metric_df_train['y'] != 0 ] # can drop zero values (due to system crash)
# c_y_train = np.asarray(metric_df_train.y)*(yMax-yMin) + yMin
# c_yhat_train = np.asarray(metric_df_train.yhat)*(yMax-yMin) + yMin
# mape_train  = np.mean(np.abs(c_y_train-c_yhat_train)/c_y_train)*100
# print('Train RMSE: %.3f' % (rmse_train*(yMax-yMin)+yMin))
# print('Train MAPE: %.3f'% mape_train)
# print('Accuracy on Training Dataset: %.3f'%(100-mape_train))

#### Denormalize the Outputs by LSTM

In [ ]:
# # UNCOMMENT to denormalize the predictions on the training dataset
# yhat_train = yhat_train*(yMax - yMin) + yMin
yhat = yhat*(yMax - yMin) + yMin

In [ ]:
# dill.load_session('/content/drive/My Drive/Colab Notebooks/sessions/lstm_80_20_entire_data.db')

In [ ]:
# plt.plot()
critical_start = datetime(2019,5,26,12,0,0)
critical_end =  datetime(2019,5,26,22,59)
holiday_start = datetime(2019,5,27,0,0,0)
holiday_end =  datetime(2019,5,27,23,59)

holiday_range = pd.date_range(start = holiday_start, end = holiday_end, freq = 'T')
# print(pd.to(holiday_range.astype(str)))

In [ ]:
# !pip install mpld3
%matplotlib notebook
import mpld3
mpld3.enable_notebook()

In [ ]:
plt.grid()
plt.plot(test_date, np.asarray(week2['Value']),'y',label = '2w' )
plt.plot(test_date, np.asarray(week1['Value']),'r',label = '1w' )
plt.plot(test_date,test_y*(yMax-yMin) + yMin,'b', label = 'current')

plt.plot(test_date, yhat[:,0],'#00ffff',label = 'lstm')
# plt.axvspan(xmin = critical_start, xmax = critical_end ,alpha=0.3,color = 'g', label = 'Zone of Concern')
# plt.axvspan(xmin = holiday_start, xmax = holiday_end ,alpha=0.2,color = 'r', label = 'Holiday')
plt.title('Using last:%d min to predict next %d mins'%(last_n, pred_next))
plt.legend()
plt.show()